In [1]:
import torch
import torch.nn as nn
import os
from PIL import Image
import ttach as tta
import yaml
import albumentations as A
import numpy as np
from albumentations.pytorch import ToTensorV2
device=torch.device("cuda:0")
config_path="predict_config.yaml"
with open(config_path) as f:
     predict_hyps = yaml.load(f, Loader=yaml.FullLoader)
path_to_images=""
segmentation_model = torch.load(predict_hyps["segmentation"]).eval().to(device)
#classification_signal = torch.load(predict_hyps["signal"]).to(device)
#classification_t_switch = torch.load(predict_hyps["tswitch"]).to(device)
#classification_f_switch = torch.load(predict_hyps["fswitch"]).to(device)
transform = A.Compose([
            A.Resize(height=1536,width=2688),
            A.Normalize([predict_hyps['mean0'], predict_hyps['mean1'], predict_hyps['mean2']], [predict_hyps['std0'],predict_hyps['std1'], predict_hyps['std2']]),
            ToTensorV2()])


In [2]:
transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.Rotate90(angles=[0, 180]),
        tta.Resize(sizes=[(320, 576), (640, 1280),(960,1920),(1536, 2688)], original_size=(1536, 2688), interpolation="nearest")  
    ]
)
#tta_model = tta.SegmentationTTAWrapper(segmentation_model, transforms,merge_mode='mean')

In [8]:
img = Image.open("/files/hackhathon_baseline/SBER_ADDITION_TRAIN/train/images/img_0.522819077873499.png").convert("RGB")
sample = {"image": img}
sample['image'] = np.array(sample['image'])[:, :, ::-1]
sample = transform(**sample)
image= sample["image"].unsqueeze(0)
orig_mask=torch.zeros((1,4,1536,2688)).to(device)
counter=0
for transformer in transforms:
    with torch.no_grad():
        augmented_image = transformer.augment_image(image).to(device)
        #print(augmented_image.size())
        model_output = segmentation_model(augmented_image)
        print(torch.argmax(torch.softmax(model_output,dim=1),dim=1))
        deaug_mask = transformer.deaugment_mask(model_output)
        orig_mask += deaug_mask
        counter+=1

tensor([[[3, 3, 3,  ..., 3, 3, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         ...,
         [0, 0, 0,  ..., 0, 0, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         [3, 3, 3,  ..., 3, 3, 3]]], device='cuda:0')
tensor([[[3, 3, 3,  ..., 3, 0, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         ...,
         [0, 0, 0,  ..., 0, 0, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         [3, 3, 3,  ..., 3, 3, 3]]], device='cuda:0')
tensor([[[3, 3, 3,  ..., 0, 0, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         ...,
         [3, 0, 0,  ..., 0, 0, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         [3, 3, 3,  ..., 3, 3, 3]]], device='cuda:0')
tensor([[[3, 3, 3,  ..., 3, 0, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         [3, 0, 0,  ..., 0, 0, 3],
         ...,
         [3, 0, 0,  ..., 0, 0, 3],
         [3, 3, 1,  ..., 0, 0, 3],
         [3, 3, 3,  ..., 3, 0, 3]]], device='cuda:0')
tensor([[[3, 0, 0,  ..., 3, 

In [6]:
print(torch.argmax(torch.softmax(orig_mask/counter,dim=1),dim=1))

tensor([[[3, 3, 3,  ..., 3, 3, 3],
         [3, 3, 3,  ..., 3, 3, 3],
         [3, 3, 0,  ..., 0, 3, 3],
         ...,
         [3, 3, 0,  ..., 0, 3, 3],
         [3, 3, 3,  ..., 3, 3, 3],
         [3, 3, 3,  ..., 3, 3, 3]]], device='cuda:0')


In [51]:
for transformer in transforms: # custom transforms or e.g. tta.aliases.d4_transform() 
    
    # augment image
    augmented_image = transformer.augment_image(img)

TypeError: rot90(): argument 'input' (position 1) must be Tensor, not Image